<a href="https://colab.research.google.com/github/aakinas/team-17/blob/main/team_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash pandas plotly flask-ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 43.0 MB/s eta 0:00:00


In [ ]:
!pip install dash-bootstrap-components


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 3.1 MB/s eta 0:00:00


In [19]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Load the datasets from CSV files for each year
years = list(range(2012, 2025))  # Years from 2012 to 2024
data = {year: pd.read_csv(f'/content/Exchange_Rate_Report_{year}.csv') for year in years}

# Convert Date columns to datetime format
for year in years:
    data[year]['Date'] = pd.to_datetime(data[year]['Date'], format='%d-%b-%y')

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app with enhanced colors
app.layout = html.Div(
    style={'padding': '20px', 'fontFamily': 'Arial, sans-serif', 'backgroundColor': '#f9f9f9'},
    children=[
        # Title
        html.H1("Exchange Rate Analyzer with Basket & Risk Indicator",
                style={'textAlign': 'center', 'color': '#333', 'fontWeight': 'bold', 'fontSize': '36px'}),

        # Currency Analyzer Section
        html.Div(
            style={'marginBottom': '40px', 'border': '1px solid #ccc', 'borderRadius': '5px', 'padding': '20px', 'backgroundColor': '#ffffff'},
            children=[
                html.H3("Currency Analyzer", style={'color': '#0066CC', 'fontWeight': 'bold'}),

                # Year Dropdown
                html.Div(
                    style={'width': '30%', 'display': 'inline-block', 'paddingRight': '10px'},
                    children=[
                        dcc.Dropdown(
                            id='year-dropdown',
                            options=[{'label': str(year), 'value': year} for year in years],
                            value=2012,  # Default value
                            style={'width': '100%', 'backgroundColor': '#e9ecef', 'color': '#333'}
                        )
                    ]
                ),

                # Currency 1 and 2 Dropdowns
                html.Div(
                    style={'width': '30%', 'display': 'inline-block', 'paddingRight': '10px'},
                    children=[
                        dcc.Dropdown(
                            id='currency-dropdown-1',
                            value=None,  # Default value set in the callback
                            style={'backgroundColor': '#e9ecef', 'color': '#333'}
                        )
                    ]
                ),
                html.Div(
                    style={'width': '30%', 'display': 'inline-block'},
                    children=[
                        dcc.Dropdown(
                            id='currency-dropdown-2',
                            value=None,  # Default value set in the callback
                            style={'backgroundColor': '#e9ecef', 'color': '#333'}
                        )
                    ]
                ),

                # Date Range Picker
                html.Div(
                    style={'marginTop': '20px'},
                    children=[
                        dcc.DatePickerRange(
                            id='date-picker-range',
                            start_date=None,  # Default value set in the callback
                            end_date=None,     # Default value set in the callback
                            style={'backgroundColor': '#e9ecef'}
                        )
                    ]
                ),

                # Granularity Dropdown
                html.Div(
                    style={'marginTop': '10px', 'width': '30%'},
                    children=[
                        dcc.Dropdown(
                            id='granularity-dropdown',
                            options=[
                                {'label': 'Daily', 'value': 'D'},
                                {'label': 'Weekly', 'value': 'W'},
                                {'label': 'Monthly', 'value': 'M'},
                                {'label': 'Yearly', 'value': 'Y'},
                            ],
                            value='D',  # Default value
                            style={'backgroundColor': '#e9ecef', 'color': '#333'}
                        )
                    ]
                ),

                # Graph for Exchange Rates
                dcc.Graph(id='exchange-rate-graph', style={'marginTop': '20px'}),
            ]
        ),

        # Custom Currency Basket Section
        html.Div(
            style={'marginBottom': '40px', 'border': '1px solid #ccc', 'borderRadius': '5px', 'padding': '20px', 'backgroundColor': '#ffffff'},
            children=[
                html.H3("Custom Currency Basket", style={'color': '#0066CC', 'fontWeight': 'bold'}),

                # Dropdown for Basket Currencies
                dcc.Dropdown(
                    id='basket-currency-dropdown',
                    multi=True,  # Allow multiple selections
                    style={'width': '50%', 'marginBottom': '10px', 'backgroundColor': '#e9ecef', 'color': '#333'}
                ),

                # Input for Basket Weights
                dcc.Input(
                    id='basket-weights',
                    type='text',
                    placeholder='Enter weights as comma-separated values (e.g. 0.5, 0.3, 0.2)',
                    style={'width': '50%', 'marginBottom': '10px', 'border': '1px solid #ccc', 'borderRadius': '5px'}
                ),

                # Base Currency Dropdown
                dcc.Dropdown(
                    id='base-currency-dropdown',
                    value=None,  # Default value set in the callback
                    style={'width': '50%', 'marginBottom': '10px', 'backgroundColor': '#e9ecef', 'color': '#333'}
                ),

                # Graph for Custom Basket
                dcc.Graph(id='basket-graph', style={'marginTop': '20px'})
            ]
        ),

        # Risk Indicator Section
        html.Div(
            style={'border': '1px solid #ccc', 'borderRadius': '5px', 'padding': '20px', 'backgroundColor': '#ffffff'},
            children=[
                html.H3("Risk Indicator (Volatility)", style={'color': '#0066CC', 'fontWeight': 'bold'}),

                # Label to display the volatility level
                html.Div(id='risk-indicator-label', style={'fontSize': '20px', 'color': '#333', 'marginBottom': '10px'}),

                # Graph for Risk Indicator
                dcc.Graph(id='risk-indicator-graph', style={'marginTop': '20px'})
            ]
        )
    ]
)

# Callbacks to update UI and graphs

# Update UI for currency dropdowns, basket dropdown, and date picker range
@app.callback(
    [Output('date-picker-range', 'start_date'),
     Output('date-picker-range', 'end_date'),
     Output('currency-dropdown-1', 'options'),
     Output('currency-dropdown-2', 'options'),
     Output('currency-dropdown-1', 'value'),
     Output('currency-dropdown-2', 'value'),
     Output('basket-currency-dropdown', 'options'),
     Output('base-currency-dropdown', 'options')],
    Input('year-dropdown', 'value')
)
def update_ui(selected_year):
    min_date = data[selected_year]['Date'].min()
    max_date = data[selected_year]['Date'].max()

    # Prepare currency options for dropdowns
    currencies = [{'label': currency, 'value': currency} for currency in data[selected_year].columns[1:]]

    return (min_date, max_date, currencies, currencies,
            currencies[0]['value'], currencies[1]['value'],
            currencies, currencies)


# Update Exchange Rate Graph
@app.callback(
    Output('exchange-rate-graph', 'figure'),
    [Input('year-dropdown', 'value'),
     Input('currency-dropdown-1', 'value'),
     Input('currency-dropdown-2', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('granularity-dropdown', 'value')]
)
def update_graph(selected_year, currency1, currency2, start_date, end_date, granularity):
    filtered_data = data[selected_year]

    if start_date is not None and end_date is not None:
        filtered_data = filtered_data[(filtered_data['Date'] >= start_date) & (filtered_data['Date'] <= end_date)]

    filtered_data['Exchange_Rate'] = filtered_data[currency1] / filtered_data[currency2]
    filtered_data.set_index('Date', inplace=True)

    if granularity == 'D':
        grouped_data = filtered_data.reset_index()
    elif granularity == 'W':
        grouped_data = filtered_data.resample('W').mean().reset_index()
    elif granularity == 'M':
        grouped_data = filtered_data.resample('M').mean().reset_index()
    elif granularity == 'Y':
        grouped_data = filtered_data.resample('Y').mean().reset_index()

    fig = px.line(grouped_data, x='Date', y='Exchange_Rate',
                  title=f'Exchange Rate: {currency1}/{currency2} ({selected_year}) - {granularity}',
                  line_shape='linear', color_discrete_sequence=['#1f77b4'])
    return fig


# Update Custom Currency Basket Graph
@app.callback(
    Output('basket-graph', 'figure'),
    [Input('year-dropdown', 'value'),
     Input('basket-currency-dropdown', 'value'),
     Input('basket-weights', 'value'),
     Input('base-currency-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_basket_graph(selected_year, selected_currencies, weight_str, base_currency, start_date, end_date):
    # Validate and parse weights
    try:
        weights = list(map(float, weight_str.split(',')))
        if len(selected_currencies) != len(weights):
            return px.line(title="Number of currencies and weights must match.")
        if sum(weights) != 1.0:
            weights = [w / sum(weights) for w in weights]  # Normalize weights to sum up to 1
    except:
        return px.line(title="Invalid weights format. Please enter valid weights.")

    filtered_data = data[selected_year]

    # Filter data by the selected date range
    if start_date is not None and end_date is not None:
        filtered_data = filtered_data[(filtered_data['Date'] >= start_date) & (filtered_data['Date'] <= end_date)]

    # Ensure all selected currencies are present in the data
    if not all(currency in filtered_data.columns for currency in selected_currencies):
        return px.line(title="Selected currencies are not available in the data.")

    # Calculate the weighted exchange rate for the basket
    basket_rate = sum(filtered_data[currency] * weight for currency, weight in zip(selected_currencies, weights))

    # Convert the basket rate to the base currency if specified
    if base_currency in filtered_data.columns:
        basket_rate = basket_rate / filtered_data[base_currency]
    else:
        return px.line(title="Base currency not found in data.")

    filtered_data['Basket_Rate'] = basket_rate

    # Plot the basket rate over time
    fig = px.line(filtered_data, x='Date', y='Basket_Rate',
                  title=f'Custom Currency Basket ({selected_year})',
                  line_shape='linear', color_discrete_sequence=['#2ca02c'])

    return fig




# Update Risk Indicator Graph
@app.callback(
    Output('risk-indicator-graph', 'figure'),
    Output('risk-indicator-label', 'children'),  # Add an output for the risk indicator label
    [Input('year-dropdown', 'value'),
     Input('currency-dropdown-1', 'value'),
     Input('currency-dropdown-2', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]
)
def update_risk_graph(selected_year, currency1, currency2, start_date, end_date):
    filtered_data = data[selected_year]

    if start_date is not None and end_date is not None:
        filtered_data = filtered_data[(filtered_data['Date'] >= start_date) & (filtered_data['Date'] <= end_date)]

    filtered_data['Exchange_Rate'] = filtered_data[currency1] / filtered_data[currency2]
    filtered_data['Rate_Change'] = filtered_data['Exchange_Rate'].pct_change()

    volatility = filtered_data['Rate_Change'].std()

    # Classify volatility
    if volatility < 0.01:  # Threshold for low volatility
        volatility_level = "Low"
        color = "#2ca02c"  # Green
    elif volatility < 0.02:  # Threshold for medium volatility
        volatility_level = "Medium"
        color = "#ff7f0e"  # Orange
    else:  # High volatility
        volatility_level = "High"
        color = "#d62728"  # Red

    fig = px.line(filtered_data, x='Date', y='Rate_Change',
                  title=f'Risk Indicator: Volatility = {volatility:.4f} ({selected_year})',
                  line_shape='linear', color_discrete_sequence=[color])

    risk_label = f"Volatility Level: {volatility_level} (Standard Deviation: {volatility:.4f})"

    return fig, risk_label  # Return the figure and the risk label


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>